In [1]:
import regionmask
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
#np.set_printoptions(edgeitems=2)

def weighted_mean(da, weights, dim):
    """Reduce da by a weighted mean along some dimension(s).

    Parameters
    ----------
    da : DataArray
        Object over which the weighted reduction operation is applied.    
    weights : DataArray
        An array of weights associated with the values in this Dataset.
    dim : str or sequence of str, optional
        Dimension(s) over which to apply the weighted `mean`.
        
    Returns
    -------
    weighted_mean : DataArray
        New DataArray with weighted mean applied to its data and
        the indicated dimension(s) removed.
    """

    weighted_sum = (da * weights).sum(dim=dim, skipna=True)
    # need to mask weights where data is not valid
    masked_weights = weights.where(da.notnull())
    sum_of_weights = masked_weights.sum(dim=dim, skipna=True)
    valid_weights = sum_of_weights != 0
    sum_of_weights = sum_of_weights.where(valid_weights)

    return weighted_sum / sum_of_weights

def average_over_AR6_region(filename, variable, region_name):

    # -- AR6 regions
    #ar6_all = regionmask.defined_regions.ar6.all
    # -- Get the regions
    ar6_land = regionmask.defined_regions.ar6.land

    #ax = ar6_all.plot()
    # -- Get land/sea mask (generic)
    land_110 = regionmask.defined_regions.natural_earth.land_110

    # -- Get data
    ds = xr.open_dataset(filename, decode_times=False)
    dat = ds[variable]
    dat.values = np.array(dat.values, dtype=np.float32)

    # -- Mask the data
    mask_3D = ar6_land.mask_3D(dat) # AR6 mask
    land_mask = land_110.mask_3D(dat) # Land sea mask
    mask_lsm = mask_3D * land_mask.squeeze(drop=True) # Combine the two

    weights = np.cos(np.deg2rad(dat.lat))
    
    if region_name=='all':
        return weighted_mean(dat, mask_3D * weights, ("lon", "lat"))
    else:
        if isinstance(region_name, list):
            res = list()
            for region in region_name:
                region_mask = mask_lsm.isel(region=list(mask_3D.abbrevs).index(region))
                dat_region = dat.where(region_mask)
                weights_region = weights.where(region_mask)
                res.append( weighted_mean(dat_region, region_mask*weights_region, ("lon","lat")) )
            return res
        else:
            region_mask = mask_lsm.isel(region=list(mask_3D.abbrevs).index(region_name))
            dat_region = dat.where(region_mask)
            weights_region = weights.where(region_mask)            
            return weighted_mean(dat_region, region_mask*weights_region, ("lon","lat"))
    

region_name = "all"
variable = 'tx35'
filename = "/data/jservon/IPCC/tx35/individual_models/CMIP6_ssp585_tx35_2100_NorESM2-LM_r1i1p1f1.nc"

tmp = average_over_AR6_region(filename, variable, region_name)
tmp

<xarray.DataArray (time: 12, region: 46)>
array([[0.      , 0.      , 0.      , ..., 0.002866, 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       ...,
       [0.      , 0.      , 0.005968, ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ]])
Coordinates:
  * time     (time) float64 0.0 31.0 59.0 90.0 120.0 ... 243.0 273.0 304.0 334.0
    member   |S24 'NorESM2-LM_r1i1p1f1'
  * region   (region) int64 0 1 2 3 4 5 6 7 8 9 ... 37 38 39 40 41 42 43 44 45
    abbrevs  (region) <U4 u'GIC' u'NWN' u'NEN' u'WNA' ... u'NZ' u'EAN' u'WAN'
    names    (region) <U22 u'Greenland/Iceland' ... u'W.Antarctica'

In [ ]:
# -- Read the CMIP6 data



# -- for baseline, GWLs, 85 mid and late, 26 mid and late
# -- Compute averages
# -- Save in json: one json file per period/GWLs

In [2]:
import numpy as np
test = [1,2,3]
np.mean(test)

2.0

In [10]:
tmp.sel(region=4)

<xarray.DataArray (time: 12)>
array([0.000000e+00, 7.005170e-02, 3.435485e-01, 5.291421e+00, 9.001981e+00,
       2.052520e+01, 2.777097e+01, 2.695861e+01, 1.489953e+01, 5.912698e+00,
       0.000000e+00, 1.635167e-02])
Coordinates:
  * time     (time) float64 0.0 31.0 59.0 90.0 120.0 ... 243.0 273.0 304.0 334.0
    member   |S24 'NorESM2-LM_r1i1p1f1'
    region   int64 4
    abbrevs  <U4 u'CNA'
    names    <U22 u'C.North-America'

In [12]:
ar6_land = regionmask.defined_regions.ar6.land
ar6_land

<regionmask.Regions>
Name:     AR6 reference regions (land only)
Source:   Iturbide et al., 2020 (Earth Syst. Sci. Data)

Regions:
  0  GIC  Greenland/Iceland
  1  NWN  N.W.North-America
  2  NEN  N.E.North-America
  3  WNA    W.North-America
  4  CNA    C.North-America
..   ...                ...
 41  EAU        E.Australia
 42  SAU        S.Australia
 43   NZ        New-Zealand
 44  EAN       E.Antarctica
 45  WAN       W.Antarctica

[46 regions]

In [13]:
ar6_land.abbrevs

[u'GIC',
 u'NWN',
 u'NEN',
 u'WNA',
 u'CNA',
 u'ENA',
 u'NCA',
 u'SCA',
 u'CAR',
 u'NWS',
 u'NSA',
 u'NES',
 u'SAM',
 u'SWS',
 u'SES',
 u'SSA',
 u'NEU',
 u'WCE',
 u'EEU',
 u'MED',
 u'SAH',
 u'WAF',
 u'CAF',
 u'NEAF',
 u'SEAF',
 u'WSAF',
 u'ESAF',
 u'MDG',
 u'RAR',
 u'WSB',
 u'ESB',
 u'RFE',
 u'WCA',
 u'ECA',
 u'TIB',
 u'EAS',
 u'ARP',
 u'SAS',
 u'SEA',
 u'NAU',
 u'CAU',
 u'EAU',
 u'SAU',
 u'NZ',
 u'EAN',
 u'WAN']

In [14]:
ar6_land.names

[u'Greenland/Iceland',
 u'N.W.North-America',
 u'N.E.North-America',
 u'W.North-America',
 u'C.North-America',
 u'E.North-America',
 u'N.Central-America',
 u'S.Central-America',
 u'Caribbean',
 u'N.W.South-America',
 u'N.South-America',
 u'N.E.South-America',
 u'South-American-Monsoon',
 u'S.W.South-America',
 u'S.E.South-America',
 u'S.South-America',
 u'N.Europe',
 u'West&Central-Europe',
 u'E.Europe',
 u'Mediterranean',
 u'Sahara',
 u'Western-Africa',
 u'Central-Africa',
 u'N.Eastern-Africa',
 u'S.Eastern-Africa',
 u'W.Southern-Africa',
 u'E.Southern-Africa',
 u'Madagascar',
 u'Russian-Arctic',
 u'W.Siberia',
 u'E.Siberia',
 u'Russian-Far-East',
 u'W.C.Asia',
 u'E.C.Asia',
 u'Tibetan-Plateau',
 u'E.Asia',
 u'Arabian-Peninsula',
 u'S.Asia',
 u'S.E.Asia',
 u'N.Australia',
 u'C.Australia',
 u'E.Australia',
 u'S.Australia',
 u'New-Zealand',
 u'E.Antarctica',
 u'W.Antarctica']

In [ ]:
Africa = [20,21,22,23,24,25,26,27,36,19]

Europe = [16,17,19,18]

# Asia 
# AustralAsia
# North America
# Central America
# South America


In [15]:
39/12.

3.25

In [17]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [20]:
import numpy as np

# Define vertices of polygon (lat/lon)
v0 = [7.5, -2.5] 
v1 = [2, 3.5]
v2 = [-2, 4]
v3 = [-5.5, -4]
v4 = [0, -10]
lats_vect = np.array([v0[0],v1[0],v2[0],v3[0],v4[0]])
lons_vect = np.array([v0[1],v1[1],v2[1],v3[1],v4[1]])

# Point of interest P
x, y = -6, 5 # x = Lat, y = Lon
x, y = 0, 2 # x = Lat, y = Lon

lons_lats_vect = np.column_stack((lons_vect, lats_vect)) # Reshape coordinates
polygon = Polygon(lons_lats_vect) # create polygon
point = Point(y,x) # create point
print(polygon.contains(point)) # check if polygon contains point
print(point.within(polygon)) # check if a point is in the polygon 

True
True


In [ ]:
# -- R